### Welcome! This notebook describes a method of generating inferential decompositions from text as introduced in our EMNLP 2023 paper: [Natural Language Decompositions of Implicit Content Enable Better Text Representations](https://arxiv.org/pdf/2305.14583.pdf)!

We will guide you through the process step-by-step, and provide explanations and code snippets along the way. The method can be broken down into the following steps:

1. **Sample a small number of items from your dataset**: Here, we use a dataset of tweets posted by legislators during the 115, 116 and 117th US Congresses. 
2. **Craft Implicit and Explicit Propositions**: Refer to Appendix 2. of our paper for a description of the instructions used in the paper to craft exemplar poropotitions. We will use the same instructions to craft implicit and explicit propositions for our dataset.
3. **Prompt an LLM with the crafted exemplars**: Here, we will use GPT3.5 Turbo for our experiments. 
4. **Validate**: Confirm that a random sample of the generated decompositions are _plausible_.
5. **Downstream Usage**: Use the decompositions in the target task. 

#### Getting Started

To begin, run the first cell below to import the necessary packages and set up the environment. The helper functions and accompanying code are in `eval_mteb.py` and `generation_utils.py`. 

##### Note: 
We assume that your OPENAI_API_KEY is an environment variable. One way to set it is by running - `conda env config vars set OPENAI_API_KEY=<your_key_here>` inside your conda environment. It can also be set manually in the config by setting  `config["llm"]["openai_api_key"]`.

The code below was tested on a linux server, but should work on other hardware with alterations to pytorch versioning. \[More additions coming soon!]

In [1]:
import os 
import json 
from tqdm import tqdm
import pandas as pd
import random
from simple_colors import * 


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
from pathlib import Path

from transformers import GenerationConfig 


#### Choosing the data 

For the purpose of this tutorial, we choose a dataset of congressional tweets sampled from the 115th, 116th and 117th Congress. The data can be found in `data/sampled_tweets_senate_115-117.jsonl`.

### Step 1: Sample a small number of items from your dataset

We sampled the following tweets from the dataset to create our exemplar propositions.

```
1. "The #HonestAds Act will strengthen protections against foreign interference in our election. No more election ads paid for in rubles."
2. "Our nation is hurting.\\n\\nGeorge Floyd's death was horrific and justice must be served. A single act of violence at the hands of an officer is one too many. \\n\\nGeorge Floyd deserved better. All black Americans do. Indeed, all Americans do."
3. "Happy Wyoming Day! Today, our great Equality State celebrates 151 years of being the first to officially recognize women's inherent right to vote and to hold office. "
4. "\"More apologies from Mark Zuckerberg won't fix Facebook. We need accountability and action \\u2013 not vague commitments to do better while continuing to profit off of users' personal data. "
5. "Finding a permanent solution to #ProtectDreamers is as urgent a task as ever. President Trump created this crisis, and he should stop tanking bipartisan congressional efforts to solve it. We owe it to these kids to keep them in the only country they've ever known as home."
6. "Qualified immunity reform should have as its focus professionalizing police departments, institutionalizing best police practices when it comes to use of force, and protecting constitutional rights of American citizens."
7. "Survivors of the coronavirus show symptoms of ME/CFS, a debilitating and chronic illness that already impacts 2.5 million Americans. I am fighting to secure the funding needed to treat this disease and give patients the care they need.
```

### Step 2: Craft Implicit and Explicit Propositions

Then we craft both explicit and implicit propositions corresponding to each of the tweets, which can be found in `exemplars/leg_tweets_exemplars.jsonl`

### DIY:  Create your own exemplars!  (Optional)

#### Step 2.1: Sample random comments from the dataset

In [2]:
from misc_utils import read_jsonl, write_jsonl, create_textboxes, show_document

TWEETS_FILEPATH = Path('data/sampled_tweets_senate_115-117.jsonl')
tweets = read_jsonl(TWEETS_FILEPATH)

random.seed(42)
exemplar_candidates = random.sample(tweets, 10)
exemplar_tweets = [x['tweet'] for x in exemplar_candidates]

In [3]:
# Display the tweets

for index, tweet in enumerate(exemplar_tweets): 
    
    text = show_document(index, tweet) 
    display(text) 

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 1:</h3><p style='font-family: Verdana'>G…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 2:</h3><p style='font-family: Verdana'>E…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 3:</h3><p style='font-family: Verdana'>O…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 4:</h3><p style='font-family: Verdana'>F…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 5:</h3><p style='font-family: Verdana'>T…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 6:</h3><p style='font-family: Verdana'>🚨…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 7:</h3><p style='font-family: Verdana'>@…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 8:</h3><p style='font-family: Verdana'>I…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 9:</h3><p style='font-family: Verdana'>S…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 10:</h3><p style='font-family: Verdana'>…

#### Step 2.2: Create your own exemplars

Once you are done writing the exemplars for a particular tweet, press the "Submit" button. 

### TIP: 
If you want to start with the exemplars used in the paper, turn the ```start_with_existing_exemplars``` flag to ```True``` in the next cell.

In [4]:
start_with_existing_exemplars = True

In [5]:
tweet_decomp_exemplars = []

if not start_with_existing_exemplars : 
    random.seed(42)
    exemplar_candidates = random.sample(tweets, 10)
    exemplar_tweets = [x['tweet'] for x in exemplar_candidates]

else: 
    # load exemplars used in the paper 
    paper_exemplars = read_jsonl("exemplars/leg_tweets_exemplars.jsonl")
    exemplar_tweets = [x[0] for x in paper_exemplars] 
    exemplar_decomps = [x[1] for x in paper_exemplars] 

for index, tweet in enumerate(exemplar_tweets[:5]): # change slicing to include all tweets
    fancy_text = show_document(index, tweet)
    
    # display the document 
    display(fancy_text)
    
    if start_with_existing_exemplars: 
        decomps = create_textboxes("\n".join(exemplar_decomps[index]))
    else: 
        decomps = create_textboxes()
    
    tweet_decomp_exemplars.append([tweet, decomps])

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 1:</h3><p style='font-family: Verdana'>T…

Textarea(value='The Honest Ads Act will strengthen protections against foreign election interference\nRussia w…

Button(description='Submit', style=ButtonStyle())

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 2:</h3><p style='font-family: Verdana'>O…

Textarea(value="A police officer killed George Floyd\nGeorge Floyd's death was unjust\nBlack Americans deserve…

...

Textarea(value="Mark Zuckerberg makes insincere apologies\nFacebook is trying to avoid accountability\nFaceboo…

Button(description='Submit', style=ButtonStyle())

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 5:</h3><p style='font-family: Verdana'>F…

Textarea(value='There is an urgent need to help DACA recipients\nSupport for Dreamers is bipartisan\nDACA rece…

Button(description='Submit', style=ButtonStyle())

#### Step 3: Save them in the right format 

In [6]:
with open("exemplars/user_collected_exemplars.jsonl", "w") as f: 
    for elem in tweet_decomp_exemplars: 
        s= json.dumps(elem)
        f.write(f"{s}\n")

### Step 3: Prompting a LLM with the crafted exemplars

We use the `GenerationEmbedder` class from `eval_mteb.py` along with the hyperparameters specified in `configs/leg-tweet-gen-gpt3.5-propositions-all.yaml` to prompt GPT3.5 Turbo with the exemplars. The generated decompositions can be found in `data/gpt3.5_tweets_to_gen_all.jsonl`. 

For your use, the decompositions will be stored in `outputs/test.jsonl`. Similar to **Step 2**, if you want to use the exemplars described in our paper, set `use_existing_exemplars = True` in the next cell.


In [7]:
use_existing_exemplars = True

In [8]:
from eval_mteb import  GenerationEmbedder, load_config

TWEETS_FILEPATH = Path('data/sampled_tweets_senate_115-117.jsonl')
tweets = read_jsonl(TWEETS_FILEPATH)

# load the config file and the exemplars 
config = load_config('configs/leg-tweet-gen-gpt3.5-propositions-all.yaml')

# use existing exemplars or 
if use_existing_exemplars is True: 
    exemplars = read_jsonl(config["data"]['exemplars_path'])
else: 
    exemplars = read_jsonl("exemplars/user_collected_exemplars.jsonl") 

# initialize the generation object with hyperparameters loaded from the config file
model = GenerationEmbedder(
    instructions=config["data"]["instructions"],
    openai_api_key=config["llm"]["openai_api_key"],
    exemplar_pool=exemplars,
    exemplar_format=config["exemplars"]["format"],
    exemplar_sep=config["exemplars"]["separator"],
    multi_output_sep=config["exemplars"]["multi_output_separator"],
    exemplars_per_prompt=config["exemplars"]["exemplars_per_prompt"],
    draws_per_pool=config["exemplars"]["draws_per_pool"],
    repeat_draws=config["exemplars"]["repeat_draws"],
    shuffles_per_draw=config["exemplars"]["shuffles_per_draw"],
    output_combination_strategy=config["embeddings"]["output_combination_strategy"],
    include_original_doc=config["embeddings"]["include_original_doc"],
    embedding_model_name=config["embeddings"]["embedding_model_name"],
    gen_model_name=config["llm"]["gen_model_name"],
    generations_per_prompt=config["llm"]["generations_per_prompt"],
    temperature=config["llm"]["temperature"],
    top_p=config["llm"]["top_p"],
    generation_kwargs=config["llm"]["generation_kwargs"],
    max_tokens=config["llm"]["max_tokens"],
    cache_db_path=config["main"]["cache_db_path"],
    dry_run=config["main"]["dry_run"],
    device=config["embeddings"]["device"],
    seed=config["main"]["seed"],
)

/fs/clip-scratch/rupak/envs/decompositions/lib/python3.10/site-packages/langchain/llms/openai.py:716: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


For the purpose of the tutorial, we are generating decompositions for the first 10 tweets, decompositions for the whole dataset can be found in `data/gpt3.5_tweets_to_gen_all.jsonl`.

In [9]:
# generate propositions from tweets 
# simple batching code that deals with breaks in connections

# use a small sample of tweets to test the generations

OUTPUT_PATH = Path("outputs/test3.jsonl") 

if not OUTPUT_PATH.is_file():
    # If it doesn't exist, create the file
    OUTPUT_PATH.touch()

tweet_texts = [tweet['tweet'] for tweet in tweets][:10] # remove [:10] to run on all tweets
propositions = read_jsonl(OUTPUT_PATH)

batch_size = 100
for index in tqdm(range(len(propositions), len(tweet_texts), batch_size)):
    batch = tweet_texts[index:index+batch_size]
    propositions.extend(model.generate_from_inputs(batch))
    write_jsonl(propositions, OUTPUT_PATH)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.36s/it]


#### Step 4: Validate

We sample some of the generated decompositions and confirm that they are _plausible_. In our paper, this was done using a human study. Please refer to Section 3 of our paper for more details. 


In [10]:
# before sampling, make sure to keep the tweet with the generations: 

for tweet_text, props in zip(tweet_texts, propositions):
    props.append(tweet_text)

# sample from the propositions
random.seed(42)
sample = random.sample(propositions, 5)

for elem in sample: 
    print(f"TWEET: \n{blue(elem[-1])}\n")
    print("PROPOSITONS:")
    for prop in elem[:-1]:
        print(green(prop))
    print("\n---------------------------\n")

TWEET: 
Cindy &amp; I are praying for all those in the path of #HurricaneIrma. We thank the brave volunteers &amp; urge all to listen to local officials.

PROPOSITONS:
Cindy and the speaker are praying for those affected by Hurricane Irma
Volunteers are acting bravely to assist
It is important to follow the guidance of local officials during the hurricane
Hurricane Irma poses a threat to people's safety

---------------------------

TWEET: 
We must do more to address mental health issues our veterans face and ensure all have access to treatment. @WSAZnews #suicidepreventionmonth  

PROPOSITONS:
Veterans face mental health challenges
Access to treatment for veterans should be improved
Suicide prevention efforts are important
Mental health support for veterans is necessary
@WSAZnews raises awareness about suicide prevention

---------------------------

TWEET: 
This project will bring hundreds of jobs &amp; millions of $ in economic growth to Northwest MT. #EmployMT #ConnectMT
 

PROPOSI

#### Step 5: Use the propositions for your own downstream task!

## Finding tweets with implicit similarity

When we embed a document through the surface form of its content, documents with a similar communicative intent but are expressed differently in their lexical forms are placed further in the embedding space. 

To find such document pairs, we can make use of the inferential decompositions we obtained above. These decompositions helps us get over the lexical choices of a communicator, and lets us focus instead on their communicative intent. Hence, documents that seem far in the embedding space are brought closer through their similar decompositions. 

Here, we show some samples of such document pairs. 

In [11]:
from misc_utils import distance_func
# NOTE: distance_func is a simple distance function that returns the minimum pairwise distance among all possible inferences of a pair of tweets. 

# tweets
tweet_texts = [x['tweet'] for x in tweets] 

# load all the decompositions 
decompositions = read_jsonl("data/gpt3.5_tweets_to_gen_all.jsonl") 

assert len(tweet_texts) == len(decompositions), "Length of documents don't match length of decompositions" 

In [12]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import torch
import numpy as np


st_model = SentenceTransformer("all-MiniLM-L6-v2") 
device = "cuda" if torch.cuda.is_available() else "cpu"

# compute embeddings and embedding similarities 
doc_embeddings = st_model.encode(tweet_texts, device=device, show_progress_bar=True) 
doc_distances = 1 - cos_sim(doc_embeddings, doc_embeddings) 

# compute embeddings of decompositions
# depending on your dataset, this might take a little bit of time
decomp_embeddings = np.array([st_model.encode(x, device=device) for x in tqdm(decompositions)], dtype="object")

Batches:   0%|          | 0/1228 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████| 39284/39284 [06:00<00:00, 108.90it/s]


In [13]:
# create a matrix of decomposition distances 
# try with only a subset of comments for speed 
doc_sample_size = 1000

from itertools import combinations 
pairs = list(combinations(range(doc_sample_size), 2)) 

decomp_dists = np.zeros((doc_sample_size, doc_sample_size)) 

for pair in tqdm(pairs): 
    index1, index2 = pair
    decomp_dists[index1, index2] = distance_func(decomp_embeddings[index1], decomp_embeddings[index2])[2]
    decomp_dists[index2, index1] = decomp_dists[index1, index2]


100%|████████████████████████████████████████████████████████████████████████| 499500/499500 [02:18<00:00, 3619.34it/s]


#### Finding tweet pairs that move closer in decomposition embedding space

In the next cell, we try to find pairs of documents (here, tweets) that move closer in embedding space once we look at their decompositions. The shift in distance is a hyperparameter, and best results can be obtained by setting a value that is meaningful for your dataset. 

Since we are using a distance function that looks at the minimum distance between all possible decompositions, the decomposition that brings the tweets closer is highlighted in <font color='red'>red</font>. The <font color='blue'>tweets</font> are colored in <font color='blue'>blue</font>, and <font color='green'>decompositions</font> in <font color='green'>green</font>.


In [14]:
# find pairs where distance is low(er) in decomposition space but high in comment space 
from nltk.tokenize import TweetTokenizer 
tt = TweetTokenizer() 

for pair in tqdm(pairs): 
    index1, index2 = pair
    doc_distance = float(doc_distances[index1, index2])
    decomp_distance = decomp_dists[index1, index2] 
    #print(doc_distance, decomp_distance, doc_distance - decomp_distance) 
    
    # find docs of similar length
    if not abs(len(tt.tokenize(tweet_texts[index1])) - len(tt.tokenize(tweet_texts[index2]))) < 10: 
        continue 

    # find docs where the shift is more than 0.5 and distance over decomposition is low 
    # change these values accoring 
    if doc_distance - decomp_distance > 0.25 and decomp_distance < 0.3: 
        print(index1, index2)
        print(f"Distance moved from {doc_distance} -> {decomp_distance}")
        a = distance_func(decomp_embeddings[index1], decomp_embeddings[index2])
        
        print(blue(tweet_texts[index1]))
        print(green(decompositions[index1]))
        print(red(decompositions[index1][a[0]]))
    
        print(blue(tweet_texts[index2]))
        print(green(decompositions[index2]))
        print(red(decompositions[index2][a[1]]))
        print("**************")
        print("\n\n")
        


  0%|▎                                                                         | 1785/499500 [00:00<02:22, 3487.17it/s]

1 102
Distance moved from 0.5520696640014648 -> 0.23937678337097168
Cindy &amp; I are praying for all those in the path of #HurricaneIrma. We thank the brave volunteers &amp; urge all to listen to local officials.
['Hurricane Irma poses a threat to residents of affected areas', 'The author is praying for people in the path of the storm', 'Local officials should be followed during natural disasters', 'Volunteers are appreciated during times of crisis']
Hurricane Irma poses a threat to residents of affected areas
Thinking of the many people in the Caribbean, Puerto Rico, USVI &amp; Florida in #Irma’s path. Please be safe and heed local officials' advice.  
['Hurricane Irma poses a threat to Caribbean islands, Puerto Rico, the US Virgin Islands, and Florida', 'Safety measures should be taken to minimize risk', 'People should follow the advice of local officials', 'Natural disasters can be dangerous', 'Floridians may need to evacuate']
Hurricane Irma poses a threat to Caribbean islands, Pu

  1%|▌                                                                         | 3622/499500 [00:01<02:13, 3701.52it/s]

3 153
Distance moved from 0.5087514519691467 -> 0.1455676555633545
Ignoring #climatechange won't prevent devastating storms. People's lives depend on all of us acknowledging that #ClimateChangeIsReal  
['Climate change is a real issue that cannot be ignored', 'Devastating storms are a consequence of climate change', 'Acknowledging climate change is crucial for saving lives', "The earth's climate is changing rapidly", 'Climate change affects human populations globally']
Climate change is a real issue that cannot be ignored
Wow. The US is now the ONLY country not to join the Paris climate accord. 
Talk about leading from behind.  
['The US did not join the Paris climate accord', 'All other countries have joined the Paris climate accord', "The US's absence from the Paris climate accord is a failure of leadership", 'Climate change is a serious issue', 'America should join the Paris climate accord']
Climate change is a serious issue
**************





  1%|▋                                                                         | 4774/499500 [00:01<02:10, 3785.99it/s]

...

943 955
Distance moved from 0.5880778431892395 -> 0.14453566074371338
In the Rochester &amp; Finger Lakes region, in 2016, there were 172 opioid-related deaths, a 45% increase from 2015.
['The opioid epidemic is affecting the Rochester & Finger Lakes region', 'Opioid overdose rates are increasing at an alarming rate', 'Many people have died as a result of opioid addiction', 'Opioid addiction is a serious problem', 'More needs to be done to address the opioid epidemic']
More needs to be done to address the opioid epidemic
.@POTUS's declaration of a Nationwide Public Health Emergency to address the opioid crisis is a reminder of its severity. My statement: 
['Donald Trump has declared a nationwide public health emergency regarding opioids', 'The opioid crisis is severe and ongoing', 'Action must be taken to address the opioid crisis', 'The US government is concerned about drug addiction']
Action must be taken to address the opioid crisis
**************



943 998
Distance moved from 0.62

100%|███████████████████████████████████████████████████████████████████████▉| 498905/499500 [02:42<00:00, 2221.67it/s]

955 998
Distance moved from 0.4798498749732971 -> 0.0958361029624939
.@POTUS's declaration of a Nationwide Public Health Emergency to address the opioid crisis is a reminder of its severity. My statement: 
['Donald Trump has declared a nationwide public health emergency regarding opioids', 'The opioid crisis is severe and ongoing', 'Action must be taken to address the opioid crisis', 'The US government is concerned about drug addiction']
Action must be taken to address the opioid crisis
#ICYMI: My column on an all hands on deck approach to fighting the opioid crisis. (via @twinfallstn)  
['The opioid crisis requires a collaborative solution', 'A team effort is required to end the opioid epidemic', 'We need to work together to prevent drug addiction', 'The opioid crisis needs attention']
The opioid crisis needs attention
**************





100%|████████████████████████████████████████████████████████████████████████| 499500/499500 [02:42<00:00, 3075.84it/s]

979 991
Distance moved from 0.6963950395584106 -> 0.24135714769363403
Last week the Administration argued that legal protections to stop insurance companies from kicking you off insurance for having a preexisting condition are unconstitutional. That's wrong. I’ve called on HHS to tell us how it plans to deal w/ coverage losses &amp; premium increases.
['The administration argues that protections against insurance company discrimination are unconstitutional', 'Insurance companies should be prevented from dropping patients for having preexisting conditions', 'Healthcare should be accessible for all', 'HHS needs to inform the public on how it will address healthcare coverage losses', 'Premium increases are a problem for Americans']
Healthcare should be accessible for all
Nevadans, have you visited @NVHealthLink yet to learn how you can #GetCovered during open enrollment? You can shop plans today &amp; also access in-person assistance at no cost. With the holidays just around the corner, d